In [16]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [17]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)
    skip = getattr(module, "skip", False)

    if skip:
        print("  Marked for skip")
        continue

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: chain
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:06<00:00, 1556.85it/s]


{'basic': {'expected_sample_burst_size': 3.004080497078735,
           'expected_sample_data_size': 16.448854678660854,
           'expected_time_to_recover': 33.1157,
           'packet_size': 5,
           'payload_bits': 63.789577742910744,
           'std_time_to_recover': 8.071537245779146},
 'derived': {'bit_efficency': 0.40830270406262853,
             'bit_efficency_ratio': 0.38525278187029566,
             'bit_efficency_sigma': 0.09988213371490573,
             'bit_redundancy': 101.78892225708924,
             'ideal_bit_efficency': 0.455612110289878,
             'ideal_bit_redundancy': 76.21894332630612,
             'ideal_packet_efficiency': 2.27806055144939,
             'ideal_packet_redundancy': 15.243788665261222,
             'ideal_time_to_recover': 28.00170421384337,
             'ideal_transmitted_bits': 140.00852106921687,
             'packet_efficiency': 1.9262639093514784,
             'packet_redundancy': 20.35778445141785,
             'permeability': 0.845

benchmark: 100%|██████████| 10000/10000 [00:07<00:00, 1328.98it/s]


{'basic': {'expected_sample_burst_size': 3.023471594259878,
           'expected_sample_data_size': 16.41556909770002,
           'expected_time_to_recover': 40.2122,
           'packet_size': 5,
           'payload_bits': 78.5102495297363,
           'std_time_to_recover': 9.427415932269033},
 'derived': {'bit_efficency': 0.4108438325258522,
             'bit_efficency_ratio': 0.39047975256134354,
             'bit_efficency_sigma': 0.09239017099855379,
             'bit_redundancy': 122.55075047026371,
             'ideal_bit_efficency': 0.46239955217208395,
             'ideal_bit_redundancy': 91.27851683247327,
             'ideal_packet_efficiency': 2.31199776086042,
             'ideal_packet_redundancy': 18.255703366494654,
             'ideal_time_to_recover': 33.95775327244191,
             'ideal_transmitted_bits': 169.78876636220957,
             'packet_efficiency': 1.9523987628067176,
             'packet_redundancy': 24.510150094052744,
             'permeability': 0.8444

...

{'basic': {'expected_sample_burst_size': 3.0027545290814706,
           'expected_sample_data_size': 16.562241762898612,
           'expected_time_to_recover': 27.381600000000002,
           'packet_size': 5,
           'payload_bits': 16.0,
           'std_time_to_recover': 10.747594216381637},
 'derived': {'bit_efficency': 0.13351544126995318,
             'bit_efficency_ratio': 0.11686680106348787,
             'bit_efficency_sigma': 0.047873737722265476,
             'bit_redundancy': 120.90800000000002,
             'ideal_bit_efficency': 0.13805489390842868,
             'ideal_bit_redundancy': 99.89592767796225,
             'ideal_packet_efficiency': 0.6902744695421433,
             'ideal_packet_redundancy': 19.97918553559245,
             'ideal_time_to_recover': 23.17918553559245,
             'ideal_transmitted_bits': 115.89592767796225,
             'packet_efficiency': 0.5843340053174394,
             'packet_redundancy': 24.181600000000003,
             'permeability': 0

benchmark: 100%|██████████| 10000/10000 [00:03<00:00, 3249.19it/s]


{'basic': {'expected_sample_burst_size': 3.0111344537815126,
           'expected_sample_data_size': 16.49453781512605,
           'expected_time_to_recover': 27.718000000000007,
           'packet_size': 5,
           'payload_bits': 16.0,
           'std_time_to_recover': 10.753774965099467},
 'derived': {'bit_efficency': 0.131912136118838,
             'bit_efficency_ratio': 0.11544844505375565,
             'bit_efficency_sigma': 0.047467912389229344,
             'bit_redundancy': 122.59000000000003,
             'ideal_bit_efficency': 0.1365239546820446,
             'ideal_bit_redundancy': 101.19555031395741,
             'ideal_packet_efficiency': 0.6826197734102231,
             'ideal_packet_redundancy': 20.239110062791482,
             'ideal_time_to_recover': 23.43911006279148,
             'ideal_transmitted_bits': 117.19555031395741,
             'packet_efficiency': 0.5772422252687782,
             'packet_redundancy': 24.518000000000008,
             'permeability': 0.8

benchmark: 100%|██████████| 10000/10000 [00:03<00:00, 2752.53it/s]


{'basic': {'expected_sample_burst_size': 3.010158493777258,
           'expected_sample_data_size': 16.592968833102862,
           'expected_time_to_recover': 27.3404,
           'packet_size': 5,
           'payload_bits': 16.0,
           'std_time_to_recover': 10.573274225139533},
 'derived': {'bit_efficency': 0.13354306715704806,
             'bit_efficency_ratio': 0.11704291085719302,
             'bit_efficency_sigma': 0.04796836529422457,
             'bit_redundancy': 120.702,
             'ideal_bit_efficency': 0.13827586294653355,
             'ideal_bit_redundancy': 99.71072245766162,
             'ideal_packet_efficiency': 0.6913793147326679,
             'ideal_packet_redundancy': 19.94214449153232,
             'ideal_time_to_recover': 23.14214449153232,
             'ideal_transmitted_bits': 115.71072245766162,
             'packet_efficiency': 0.5852145542859651,
             'packet_redundancy': 24.1404,
             'permeability': 0.8464449858645932,
             'sa

### LT vs ANFEXT2L benchmark sweep
This section benchmarks LT and `anfext2l_undegenerate` across multiple payload sizes, caches the distributions, and plots the expected time-to-recover (TTR) and bit efficiency per payload.

In [19]:
from __future__ import annotations

import importlib
import json
from datetime import datetime
from pathlib import Path
import math
import numpy as np

import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
from tqdm.auto import tqdm

import impls.systematic as systematic
import impls.raptor as raptor
from benchmark_utils import benchmark, compute_distribution_stats

# Always reload to pick up local code edits when rerunning the cell
importlib.reload(systematic)
importlib.reload(raptor)

# -----------------------------------------------------------------------------
# Configuration (adjust N or payload grids if you want to re-run with new params)
# -----------------------------------------------------------------------------
N = 7


def _payload_sizes_for_anfext2l(
    n: int, min_bits: int, max_bits: int, max_payload: int | None = None
) -> list[int]:
    base = (1 << n) - 2
    sizes: list[int] = []
    m = 1
    while True:
        payload = base**m
        bits = math.log2(payload)
        if bits > max_bits or (max_payload is not None and payload > max_payload):
            break
        if bits >= min_bits:
            sizes.append(payload)
        m += 1
    return sizes


ANF_OVERRIDE_FN = getattr(systematic, "override_D", None)
ANF_MAX_PAYLOAD = ANF_OVERRIDE_FN(N) if ANF_OVERRIDE_FN is not None else None
LT_OVERRIDE_FN = getattr(raptor, "override_D", None)

PAYLOAD_BITS_BY_SCHEME = {
    "raptor": [1 << bits for bits in range(N, 2 ** (N - 1) + 1)],
    "systematic": [1 << bits for bits in range(N, 2 ** (N - 1) + 1)],
}
BENCHMARK_CFG = dict(passes=200, iters_bound=400, processes=None, progress_update=100)
PLOT_WIDTH = 1050  # 1.5x wider plots
RESULTS_PATH = Path("research/lt_vs_anfext2l_results.jsonl")
RESULTS_PATH.parent.mkdir(parents=True, exist_ok=True)

SCHEMES = {
    "raptor": {
        "producer": raptor.producer_constructor,
        "recoverer": raptor.recoverer_constructor,
        "override": LT_OVERRIDE_FN,
    },
    "systematic": {
        "producer": systematic.producer_constructor,
        "recoverer": systematic.recoverer_constructor,
        "override": ANF_OVERRIDE_FN,
    },
}


def _rgba_from_hex(hex_color: str, alpha: float) -> str:
    hex_color = hex_color.lstrip("#")
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    return f"rgba({r}, {g}, {b}, {alpha})"


SCHEME_COLORS = {
    "raptor": "#1f77b4",
    "systematic": "#d62728",
}

SCHEME_SIGMA_FILLS = {
    scheme: {
        "1sigma": _rgba_from_hex(color, 0.35),
        "2sigma": _rgba_from_hex(color, 0.18),
    }
    for scheme, color in SCHEME_COLORS.items()
}


def _load_records(path: Path) -> list[dict]:
    if not path.exists():
        return []
    records: list[dict] = []
    with path.open("r", encoding="utf-8") as fh:
        for line in fh:
            line = line.strip()
            if line:
                records.append(json.loads(line))
    return records


def _append_record(path: Path, record: dict) -> None:
    with path.open("a", encoding="utf-8") as fh:
        fh.write(json.dumps(record) + "\n")


records = _load_records(RESULTS_PATH)
existing_keys = {(rec["scheme"], rec["D"], rec["N"]) for rec in records}
plan = [
    (scheme, payload_size)
    for scheme, payloads in PAYLOAD_BITS_BY_SCHEME.items()
    for payload_size in payloads
]
pending = [
    (scheme, payload_size)
    for scheme, payload_size in plan
    if (scheme, payload_size, N) not in existing_keys
]

if pending:
    total_jobs = len(pending)
    print(f"Running {total_jobs} new benchmarks (N={N}).")
    with tqdm(total=total_jobs, desc=f"Benchmarks (N={N})") as bar:
        for scheme, payload_size in pending:
            impl = SCHEMES[scheme]
            D = payload_size
            payload_bits = math.log2(D)
            override = impl.get("override")
            if override is not None:
                max_d = override(N)
                if D > max_d:
                    raise ValueError(
                        f"Payload size D={D} (bits={payload_bits:.2f}) exceeds {scheme} support for N={N}"
                    )
            bench_result = benchmark(
                impl["producer"],
                impl["recoverer"],
                N,
                D,
                **BENCHMARK_CFG,
            )
            record = {
                "scheme": scheme,
                "payload_bits": payload_bits,
                "N": N,
                "D": D,
                "timestamp": datetime.utcnow().isoformat(),
                "benchmark_cfg": BENCHMARK_CFG,
                "result": bench_result,
            }
            records.append(record)
            _append_record(RESULTS_PATH, record)
            bar.update(1)
else:
    print(f"All requested payload benchmarks already cached for N={N}.")

records_df = pd.DataFrame(records)
records_df = records_df[records_df["N"] == N].copy()
if records_df.empty:
    raise RuntimeError(
        "No cached benchmark rows for the current N; run the cell again to generate them."
    )

summary_rows = []
for rec in records_df.to_dict("records"):
    stats = compute_distribution_stats(rec["N"], rec["D"], rec["result"])
    metrics = stats["metrics"]
    summary_rows.append(
        {
            "scheme": rec["scheme"],
            "payload_bits": rec["payload_bits"],
            "expected_ttr": metrics["basic"]["expected_time_to_recover"],
            "ttr_sigma": metrics["basic"].get("std_time_to_recover", float("nan")),
            "bit_efficiency": metrics["derived"]["bit_efficency"],
            "bit_efficiency_sigma": metrics["derived"].get(
                "bit_efficency_sigma", float("nan")
            ),
            "bit_redundancy": metrics["derived"].get("bit_redundancy", float("nan")),
            "ideal_bit_efficency": metrics["derived"].get(
                "ideal_bit_efficency", float("nan")
            ),
            "ideal_bit_redundancy": metrics["derived"].get(
                "ideal_bit_redundancy", float("nan")
            ),
            "ideal_packet_efficiency": metrics["derived"].get(
                "ideal_packet_efficiency", float("nan")
            ),
            "ideal_packet_redundancy": metrics["derived"].get(
                "ideal_packet_redundancy", float("nan")
            ),
            "ideal_time_to_recover": metrics["derived"].get(
                "ideal_time_to_recover", float("nan")
            ),
            "ideal_transmitted_bits": metrics["derived"].get(
                "ideal_transmitted_bits", float("nan")
            ),
            "packet_efficiency": metrics["derived"].get(
                "packet_efficiency", float("nan")
            ),
            "packet_redundancy": metrics["derived"].get(
                "packet_redundancy", float("nan")
            ),
        }
    )

summary_df = pd.DataFrame(summary_rows).sort_values(["scheme", "payload_bits"])
print("Cached benchmark summary:")
display(summary_df)

fig_ttr = go.Figure()
for scheme, group in summary_df.groupby("scheme"):
    group = group.sort_values("payload_bits")
    payload_bits_arr = group["payload_bits"].to_numpy(dtype=float)
    expected_arr = group["expected_ttr"].to_numpy(dtype=float)
    sigma_arr = group["ttr_sigma"].to_numpy(dtype=float)
    fills = SCHEME_SIGMA_FILLS.get(scheme, {})
    if payload_bits_arr.size and not np.all(np.isnan(sigma_arr)):
        sigma_arr = np.nan_to_num(sigma_arr, nan=0.0)
        for mult, fill_key in ((2, "2sigma"), (1, "1sigma")):
            fill_color = fills.get(fill_key)
            if fill_color is None:
                continue
            upper = expected_arr + mult * sigma_arr
            lower = np.clip(expected_arr - mult * sigma_arr, a_min=0, a_max=None)
            band_label = f"{scheme} +/- {mult} sigma"
            fig_ttr.add_trace(
                go.Scatter(
                    x=np.concatenate([payload_bits_arr, payload_bits_arr[::-1]]),
                    y=np.concatenate([upper, lower[::-1]]),
                    fill="toself",
                    fillcolor=fill_color,
                    line=dict(color="rgba(0,0,0,0)"),
                    hoverinfo="skip",
                    showlegend=bool(mult == 1),
                    legendgroup=f"{scheme}_bands",
                    name=band_label,
                )
            )
    color = SCHEME_COLORS.get(scheme)
    fig_ttr.add_trace(
        go.Scatter(
            x=group["payload_bits"],
            y=group["expected_ttr"],
            mode="lines+markers",
            name=f"{scheme} E[TTR]",
            line=dict(shape="spline", color=color),
            marker=dict(color=color),
            legendgroup=scheme,
            hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>E[TTR]=%{y:.2f} packets<extra></extra>",
            text=[scheme] * len(group),
        )
    )
fig_ttr.update_layout(
    title=f"Expected time-to-recover vs payload size (N={N})",
    xaxis_title="payload bits",
    yaxis_title="expected TTR (packets)",
    width=PLOT_WIDTH,
    template="plotly_white",
)
fig_ttr.show()

fig_eff = go.Figure()
for scheme, group in summary_df.groupby("scheme"):
    group = group.sort_values("payload_bits")
    payload_bits_arr = group["payload_bits"].to_numpy(dtype=float)
    eff_arr = group["bit_efficiency"].to_numpy(dtype=float)
    sigma_arr = group["bit_efficiency_sigma"].to_numpy(dtype=float)
    fills = SCHEME_SIGMA_FILLS.get(scheme, {})
    if payload_bits_arr.size and not np.all(np.isnan(sigma_arr)):
        sigma_arr = np.nan_to_num(sigma_arr, nan=0.0)
        for mult, fill_key in ((2, "2sigma"), (1, "1sigma")):
            fill_color = fills.get(fill_key)
            if fill_color is None:
                continue
            upper = eff_arr + mult * sigma_arr
            lower = np.clip(eff_arr - mult * sigma_arr, a_min=0, a_max=None)
            band_label = f"{scheme} +/- {mult} sigma (bit eff)"
            fig_eff.add_trace(
                go.Scatter(
                    x=np.concatenate([payload_bits_arr, payload_bits_arr[::-1]]),
                    y=np.concatenate([upper, lower[::-1]]),
                    fill="toself",
                    fillcolor=fill_color,
                    line=dict(color="rgba(0,0,0,0)"),
                    hoverinfo="skip",
                    showlegend=bool(mult == 1),
                    legendgroup=f"{scheme}_eff_bands",
                    name=band_label,
                )
            )
    color = SCHEME_COLORS.get(scheme)
    fig_eff.add_trace(
        go.Scatter(
            x=group["payload_bits"],
            y=group["bit_efficiency"],
            mode="lines+markers",
            name=f"{scheme} bit efficiency",
            line=dict(shape="spline", color=color),
            marker=dict(color=color),
            legendgroup=scheme,
            hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>efficiency=%{y:.4f}<extra></extra>",
            text=[scheme] * len(group),
        )
    )
fig_eff.update_layout(
    title=f"Bit efficiency vs payload size (N={N})",
    xaxis_title="payload bits",
    yaxis_title="payload bits per transmitted bit",
    width=PLOT_WIDTH,
    template="plotly_white",
)
fig_eff.show()

ADDITIONAL_METRICS = [
    ("bit_redundancy", "Bit redundancy vs payload size", "extra bits"),
    (
        "ideal_bit_efficency",
        "Ideal bit efficiency vs payload size",
        "ideal bits per transmitted bit",
    ),
    (
        "ideal_bit_redundancy",
        "Ideal bit redundancy vs payload size",
        "ideal extra bits",
    ),
    (
        "ideal_packet_efficiency",
        "Ideal packet efficiency vs payload size",
        "payload bits per ideal packet",
    ),
    (
        "ideal_packet_redundancy",
        "Ideal packet redundancy vs payload size",
        "ideal extra packets",
    ),
    ("ideal_time_to_recover", "Ideal time-to-recover vs payload size", "ideal packets"),
    (
        "ideal_transmitted_bits",
        "Ideal transmitted bits vs payload size",
        "ideal transmitted bits",
    ),
    (
        "packet_efficiency",
        "Packet efficiency vs payload size",
        "payload bits per packet",
    ),
    ("packet_redundancy", "Packet redundancy vs payload size", "extra packets"),
]

for metric_key, title, y_axis in ADDITIONAL_METRICS:
    if metric_key not in summary_df.columns:
        continue
    fig = go.Figure()
    for scheme, group in summary_df.groupby("scheme"):
        group = group.sort_values("payload_bits")
        color = SCHEME_COLORS.get(scheme)
        fig.add_trace(
            go.Scatter(
                x=group["payload_bits"],
                y=group[metric_key],
                mode="lines+markers",
                name=f"{scheme} {metric_key}",
                line=dict(shape="spline", color=color),
                marker=dict(color=color),
                legendgroup=scheme,
                hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>value=%{y:.4f}<extra></extra>",
                text=[scheme] * len(group),
            )
        )
    fig.update_layout(
        title=f"{title} (N={N})",
        xaxis_title="payload bits",
        yaxis_title=y_axis,
        template="plotly_white",
        width=PLOT_WIDTH,
    )
    fig.show()

Running 58 new benchmarks (N=7).


Benchmarks (N=7): 100%|██████████| 58/58 [01:48<00:00,  1.88s/it]


Cached benchmark summary:


,scheme,payload_bits,expected_ttr,ttr_sigma,bit_efficiency,bit_efficiency_sigma,bit_redundancy,ideal_bit_efficency,ideal_bit_redundancy,ideal_packet_efficiency,ideal_packet_redundancy,ideal_time_to_recover,ideal_transmitted_bits,packet_efficiency,packet_redundancy
58,anfext2l_undegenerate,6.97728,7.925,3.925478,0.158077,0.079548,48.49772,0.147417,40.352940,1.031919,5.764706,6.761460,47.330220,0.880414,6.928246
59,anfext2l_undegenerate,13.95456,10.885,5.055865,0.222764,0.101163,62.24044,0.216986,50.356357,1.518901,7.193765,9.187274,64.310917,1.281999,8.891491
60,anfext2l_undegenerate,20.93184,12.740,5.110029,0.272192,0.106640,68.24816,0.275537,55.035639,1.928758,7.862234,10.852497,75.967479,1.643002,9.749737
61,anfext2l_undegenerate,27.90912,14.395,6.147274,0.320728,0.120043,72.85588,0.324931,57.983335,2.274517,8.283334,12.270351,85.892454,1.938807,10.407983
62,anfext2l_undegenerate,34.88640,15.880,6.663002,0.364990,0.136356,76.27360,0.371218,59.091811,2.598526,8.441687,13.425459,93.978211,2.196877,10.896229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,systematic,60.00000,110.070,28.280472,0.082892,0.020838,710.49000,0.092216,590.647366,0.645511,84.378195,92.949624,650.647366,0.545108,101.498571
240,systematic,61.00000,115.300,25.987689,0.079414,0.017801,746.10000,0.089066,623.887144,0.623460,89.126735,97.841021,684.887144,0.529055,106.585714
241,systematic,62.00000,115.515,27.785784,0.080975,0.018998,746.60500,0.090647,621.968209,0.634532,88.852601,97.709744,683.968209,0.536727,106.657857
242,systematic,63.00000,155.540,40.330118,0.061769,0.016105,1025.78000,0.068763,853.196929,0.481338,121.885276,130.885276,916.196929,0.405041,146.540000
